In [2]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import numpy as np
import pandas as pd
import re
import os
import random
from collections import Counter

import tensorflow as tf
from tensorflow import gfile

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
train_data_pattern = 'tfrecords/train_data_old/*.tfrecord'
train_data_folder_new = 'tfrecords/train_data_new'
test_data_pattern = 'tfrecords/test_data_old/*.tfrecord'
test_data_folder_new = 'tfrecords/test_data_new'

In [4]:
## Keep list of all new labels (to see which ones occur often)
all_labels = []
# Maximal proportion for labels
max_prop = 0.002

In [5]:
## Open old csv file and alternated csv file with the super classes
mid_to_label_old = pd.read_csv('csv_files/class_labels_indices.csv')
mid_to_label_new = pd.read_csv('csv_files/class_labels_indices_proportions.csv', sep=";")

In [6]:
## Deze functie checkt of een example wordt gemaakt (met kans)
def checkIfNewExample(labels):
    
    ## Voor het gemak: bij meer dan 1 label, return true
    if len(labels)>1:
        return True
    else:
        label = labels[0]
        proportion = mid_to_label_new.loc[np.int(label), "proportion"]
        
        ## Als proportie niet max_prop overschrijdt, return true
        if proportion < max_prop:
            return True
        else:
            prob = max_prop/proportion
            rand = random.random()
            if rand <= prob:
                return True
            else:
                return False
    return False

In [7]:
#### Find new label for each of old label (527 -> 31)

def getNewLabel(label_old):
    ## Get m-id that matches with index
    mid = mid_to_label_old.loc[mid_to_label_old['index']==label_old, 'mid']

    ## Find the new label belonging to this m-id
    label_new = mid_to_label_new.loc[mid_to_label_new['mid']==str(mid.values[0]), 'index']
    
    return label_new

In [8]:
def createExample(example_old, new_labels):
    
    #print("New labels: " + str(new_labels))
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})

    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [9]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## One tf-record has several examples
## This function checks for each example if it has a label we are interested in
## If so, we compute the one-hot vector
## Input: filename of tf-record
## Result: write new tf-record

def readWriteTfRecord(tfrecords_filename, new_folder):
    
    ## Create writer that writes new tf-record in train_data_folder_new
    path_tfrecord_new = str(new_folder + '/' + tfrecords_filename.split('\\')[-1])
    writer = tf.python_io.TFRecordWriter(path_tfrecord_new)
    nr_new_examples = 0
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        
        new_labels = []
        for label in labels:
            ## Hij returnt een set van cellen (of leeg of van 1 rij)
            label_new = getNewLabel(label)
            if(label_new.shape[0]>0):
                new_labels.append(label_new.values[0])
        
        ## If any of the labels left (for our model), write new tf-record
        if len(new_labels) > 0 and checkIfNewExample(new_labels):
        
            ## Add to all labels
            all_labels.extend(new_labels)
            
            sequence_example = createExample(example, new_labels)
            nr_new_examples += 1
            
            ## Write the example
            writer.write(sequence_example.SerializeToString())
            #print('Example gemaakt! Video-id: ' + str(example.context.feature['video_id'].bytes_list.value))

    #print("nr new examples: " + str(nr_new_examples))
    # Only write if any examples created

    if nr_new_examples > 0:
        writer.close()
        #print("tfrecord written")
    else:
        #print("empty file.. remove: " + path_tfrecord_new)
        os.remove(path_tfrecord_new)

In [10]:
## Read and rewrite all train files
#train_data_pattern = 'tfrecords/train_data_old/*2.tfrecord'
files = gfile.Glob(train_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file, train_data_folder_new)

In [11]:
cnt_labels = Counter(all_labels)
for i in cnt_labels.most_common(100):
    print(mid_to_label_new.loc[np.int(i[0])]['display_name'], ' : ', str(i[1]))

Speech  :  769
Music  :  722
Vehicle  :  250
Animal  :  226
Musical instrument  :  185
Domestic animals, pets  :  160
Car  :  111
Gunshot, gunfire  :  63
Emergency vehicle  :  47
Rail transport  :  43
Aircraft  :  36
Idling  :  29
Boat, Water vehicle  :  29
Ambulance (siren)  :  28
Medium engine (mid frequency)  :  27
Car passing by  :  27
Motor vehicle (road)  :  26
Engine starting  :  25
Jackhammer  :  25
Machine gun  :  24
Motorcycle  :  23
Train wheels squealing  :  22
Heavy engine (low frequency)  :  21
Traffic noise, roadway noise  :  21
Fire engine, fire truck (siren)  :  21
Accelerating, revving, vroom  :  19
Police car (siren)  :  19
Hammer  :  19
Chainsaw  :  19
Tools  :  19
Bus  :  19
Conversation  :  18
Power tool  :  17
Vehicle horn, car horn, honking  :  15


In [ ]:
## Read and rewrite all test files
files = gfile.Glob(test_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readWriteTfRecord(file, test_data_folder_new)

In [5]:
###########################################################################################
# Helper: show Examples in tfrecord file (only run this if you want to see just one example)
#filename = 'tfrecords/train_data_new/-1.tfrecord'
filename = 'tfrecords/added_data/gun_train.tfrecord'
record_iterator = tf.python_io.tf_record_iterator(path=filename)

for string_record in record_iterator:

    ## Take one example
    example = tf.train.SequenceExample()
    example.ParseFromString(string_record)
    print(example)
    break;

context {
  feature {
    key: "labels"
    value {
      int64_list {
        value: 15
        value: 35
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "-1LrH01Ei1w"
      }
    }
  }
}
feature_lists {
  feature_list {
    key: "audio_embedding"
    value {
      feature {
        bytes_list {
          value: "\tQ\315\360\033\221\327?\177\354\372#\327\373\374\277\306\352\326&\223\312\343? \276S\205\375&\347\277\366\027\301\n\352b\372?\000-\240U67\350\277`\203\\\373\362:\333\277>Y{`\031\033\322\277|\363\3052\'T\345?\344\341\334\025\240l\360?\021;}\tf\245\344\277\026\010\221\234\2043\340\277\332p+I\206G\360?\242`g\356-Q\326?2\371\352@\013\236\004\300\330\001\n\307.\266\362\277\264[l\325:s\361?(\024zi\025\020\347\277\000\002f\366UvR?\016\227\350\327\000C\361?\005t\204\204\260\373\372\277\006\027\251\301\372\270\001@\236Sd\227\356}\344\277\267\342\206\337\277\202\321\277\250s\271\016\317\305\325\277\237\336\270\024]C\322?\210\331i+KW\327\